# DM Final Project Review + Business Data Analysis

## Setting

In [ ]:
import json
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

## Data Loading

In [ ]:
# Specify the encoding to be 'utf-8'
with open("data/yelp_academic_dataset_business.json", encoding='utf-8') as data_file:
    data = [json.loads(line) for line in data_file]

bus_df = pd.DataFrame(data)

In [ ]:
with open("data/yelp_academic_dataset_review.json", encoding='utf-8') as data_file:
    data = [json.loads(line) for line in data_file]

review_df = pd.DataFrame(data)

In [ ]:
with open("data/yelp_academic_dataset_user.json", encoding='utf-8') as data_file:
    data = [json.loads(line) for line in data_file]

user_df = pd.DataFrame(data)

In [ ]:
with open("data/yelp_academic_dataset_checkin.json", encoding='utf-8') as data_file:
    data = [json.loads(line) for line in data_file]

checkin_df = pd.DataFrame(data)

In [ ]:
with open("data/yelp_academic_dataset_tip.json", encoding='utf-8') as data_file:
    data = [json.loads(line) for line in data_file]

tip_df = pd.DataFrame(data)

## Data Info

In [ ]:
bus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [ ]:
# Filter rows that contain the keyword 'Restaurants'
filtered_df = bus_df[bus_df['categories'].str.contains("Restaurants", case=False, na=False)]
filtered_df = filtered_df[filtered_df.city == "Santa Barbara"].reset_index(drop=True)

In [ ]:
filtered_df.shape

(767, 14)

## Attributes Cleaning

In [ ]:
from collections import Counter
import pandas as pd
import ast  # Import the ast module for safe evaluation of strings as Python expressions

# Initialize Counter objects once
attribute_counts_corrected = Counter()
true_counts_corrected = Counter()
false_counts_corrected = Counter()

# Assume df.attributes is your dataset
for item in filtered_df.attributes:
    # Ensure item is a dictionary
    if isinstance(item, dict):
        for key, value in item.items():
            # Count occurrence of each attribute (key)
            attribute_counts_corrected[key] += 1

            # Check if value is a string that represents a dictionary
            if isinstance(value, str) and value.startswith('{') and value.endswith('}'):
                try:
                    nested_dict = ast.literal_eval(value)  # Safely evaluate string as a Python dictionary
                    for nested_key, nested_value in nested_dict.items():
                        if nested_value is True:
                            true_counts_corrected[nested_key] += 1
                        elif nested_value is False:
                            false_counts_corrected[nested_key] += 1
                except ValueError as e:
                    print(f"Error parsing string to dict: {e}")
            else:
                # Directly count True/False values for non-nested attributes
                if value in ['True', True]:
                    true_counts_corrected[key] += 1
                elif value in ['False', False]:
                    false_counts_corrected[key] += 1

# Prepare the corrected summary table
summary_data_corrected = {
    'Attribute': list(attribute_counts_corrected.keys()),
    'Total Counts': list(attribute_counts_corrected.values()),
    'True Counts': [true_counts_corrected.get(key, 0) for key in attribute_counts_corrected.keys()],
    'False Counts': [false_counts_corrected.get(key, 0) for key in attribute_counts_corrected.keys()],
}

summary_df_corrected = pd.DataFrame(summary_data_corrected)

# Assuming you want to see the dataframe or use it later
print(summary_df_corrected)


                     Attribute  Total Counts  True Counts  False Counts
0           RestaurantsTakeOut           700          610            57
1                   NoiseLevel           556            0             0
2                       Caters           559          300           259
3                     Ambience           619            0             0
4      RestaurantsReservations           658          272           384
5   BusinessAcceptsCreditCards           726          711            15
6      RestaurantsTableService           335          197           138
7                  GoodForKids           615          471           144
8       RestaurantsPriceRange2           690            0             0
9         WheelchairAccessible           287          282             5
10              OutdoorSeating           703          510           155
11         RestaurantsDelivery           679          296           318
12                       HasTV           622          333       

In [ ]:
rest = filtered_df.copy(deep = True)

In [ ]:
rest[rest.business_id == "yPSejq3_erxo9zdVYTBnZA"]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
659,yPSejq3_erxo9zdVYTBnZA,Los Agaves,600 N Milpas St,Santa Barbara,CA,93101,34.427494,-119.686624,4.5,3834,1,"{'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True', 'NoiseLevel': 'u'average'', 'GoodForKids': 'True', 'RestaurantsAttire': 'u'casual'', 'RestaurantsPriceRange2': '2', 'RestaurantsGoodForGroups': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'BikeParking': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'HappyHour': 'True', 'RestaurantsTableService': 'False', 'DogsAllowed': 'F...","Mexican, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21:0', 'Thursday': '11:0-21:0', 'Friday': '11:0-21:0', 'Saturday': '11:0-21:0', 'Sunday': '11:0-21:0'}"


In [ ]:
import pandas as pd
import ast  # Import the ast module for safe evaluation

# Assuming `df` is your original DataFrame and it has a column 'attributes' containing nested dictionaries
# Normalize 'attributes' to create an initial attributes DataFrame, handling strings properly
rest['attributes'] = rest['attributes'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) else x)
attributes_df = pd.json_normalize(rest['attributes'])

In [ ]:
# Function to normalize and flatten nested dictionaries, with adjustments to keep NaN values
def flatten_nested_columns(df, column_name):
    # Safely evaluate strings to dictionaries or pass through NaN and already evaluated dictionaries
    def safe_eval(cell):
        if pd.isna(cell):
            return cell  # Keep NaN values as is
        try:
            # Safely evaluate the string as a Python literal if not NaN and if it's a string
            return ast.literal_eval(cell) if isinstance(cell, str) else cell
        except ValueError:
            # If evaluation fails, return the cell as is (might indicate a parsing error or non-dictionary string)
            return cell

    # Check if column_name exists in df to avoid KeyErrors
    if column_name in df.columns:
        # Apply safe_eval to the column, then normalize and create a new DataFrame
        normalized_data = df[column_name].apply(safe_eval)
        flattened = pd.json_normalize(normalized_data.dropna().apply(lambda x: {} if pd.isna(x) else x))

        # Prefix the columns with the original nested column name for clarity
        flattened.columns = [f"{column_name}_{sub_col}" for sub_col in flattened.columns]

        # Since we're not dropping the original column, we prepare it to be combined back
        df = df.drop(columns=[column_name], errors='ignore')

        # Join the flattened DataFrame back to the original DataFrame
        df = df.join(flattened, how='left')

    return df

# List of nested dictionary columns you identified in attributes_df
nested_columns = ['BusinessParking', 'Ambience', 'GoodForMeal', 'BestNights', 'Music']

# Apply the function to each nested dictionary column
for column in nested_columns:
    attributes_df = flatten_nested_columns(attributes_df, column)

# Merge or assign the modified attributes_df back into the original df as needed
# For example, if using direct assignment and assuming the row order hasn't changed
for column in attributes_df.columns:
    if column not in rest.columns:  # To avoid overwriting existing data
        rest[column] = attributes_df[column].values


In [ ]:
filtered_df[filtered_df.business_id == "yPSejq3_erxo9zdVYTBnZA"].attributes

659    {'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True', 'NoiseLevel': 'u'average'', 'GoodForKids': 'True', 'RestaurantsAttire': 'u'casual'', 'RestaurantsPriceRange2': '2', 'RestaurantsGoodForGroups': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'BikeParking': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'HappyHour': 'True', 'RestaurantsTableService': 'False', 'DogsAllowed': 'F...
Name: attributes, dtype: object

In [ ]:
rest[rest.business_id == "yPSejq3_erxo9zdVYTBnZA"]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,RestaurantsTakeOut,NoiseLevel,Caters,RestaurantsReservations,BusinessAcceptsCreditCards,RestaurantsTableService,GoodForKids,RestaurantsPriceRange2,WheelchairAccessible,OutdoorSeating,RestaurantsDelivery,HasTV,RestaurantsAttire,Alcohol,DogsAllowed,RestaurantsGoodForGroups,HappyHour,BikeParking,WiFi,ByAppointmentOnly,CoatCheck,BusinessAcceptsBitcoin,BYOB,GoodForDancing,DriveThru,Smoking,Corkage,BYOBCorkage,AgesAllowed,AcceptsInsurance,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,Ambience_touristy,Ambience_hipster,Ambience_romantic,Ambience_divey,Ambience_intimate,Ambience_trendy,Ambience_upscale,Ambience_classy,Ambience_casual,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_brunch,GoodForMeal_breakfast,BestNights_monday,BestNights_tuesday,BestNights_wednesday,BestNights_thursday,BestNights_friday,BestNights_saturday,BestNights_sunday,Music_dj,Music_live,Music_jukebox,Music_video,Music_background_music,Music_karaoke,Music_no_music
659,yPSejq3_erxo9zdVYTBnZA,Los Agaves,600 N Milpas St,Santa Barbara,CA,93101,34.427494,-119.686624,4.5,3834,1,"{'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True', 'NoiseLevel': 'u'average'', 'GoodForKids': 'True', 'RestaurantsAttire': 'u'casual'', 'RestaurantsPriceRange2': '2', 'RestaurantsGoodForGroups': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'BikeParking': 'True', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}', 'HappyHour': 'True', 'RestaurantsTableService': 'False', 'DogsAllowed': 'F...","Mexican, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21:0', 'Thursday': '11:0-21:0', 'Friday': '11:0-21:0', 'Saturday': '11:0-21:0', 'Sunday': '11:0-21:0'}",True,u'average',True,False,True,False,True,2,NaN,True,True,True,u'casual',u'full_bar',False,True,True,True,u'no',NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Function that extract keys from the nested dictionary
def extract_keys(attr, key):
    if attr == None:
        return "{}"
    if key in attr:
        return attr.pop(key)

# convert string to dictionary
import ast
def str_to_dict(attr):
    if attr != None:
        return ast.literal_eval(attr)
    else:
        return ast.literal_eval("{}")

In [ ]:
# get dummies from nested attributes
rest['BusinessParking'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'BusinessParking')), axis=1)
rest['Ambience'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Ambience')), axis=1)
rest['GoodForMeal'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'GoodForMeal')), axis=1)
rest['Dietary'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Dietary')), axis=1)
rest['Music'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Music')), axis=1)

In [ ]:
# create table with attribute dummies
df_attr = pd.concat([ rest['attributes'].apply(pd.Series), rest['BusinessParking'].apply(pd.Series),
                    rest['Ambience'].apply(pd.Series), rest['GoodForMeal'].apply(pd.Series),
                    rest['Dietary'].apply(pd.Series) ], axis=1)
df_attr_dummies = pd.get_dummies(df_attr)
df_attr_dummies

,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',Caters_False,Caters_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_True,RestaurantsTableService_False,RestaurantsTableService_True,GoodForKids_False,GoodForKids_True,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,WheelchairAccessible_False,WheelchairAccessible_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,HasTV_False,HasTV_True,RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',DogsAllowed_False,DogsAllowed_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,HappyHour_False,HappyHour_True,BikeParking_False,BikeParking_True,WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid',ByAppointmentOnly_False,ByAppointmentOnly_True,CoatCheck_False,CoatCheck_True,"BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday

In [ ]:
# Concat all tables and drop Restaurant column
df_final = pd.concat([filtered_df, df_attr_dummies], axis=1)
df_final.head()
# df_final.drop('Restaurants',inplace=True,axis=1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',Caters_False,Caters_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_True,RestaurantsTableService_False,RestaurantsTableService_True,GoodForKids_False,GoodForKids_True,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,WheelchairAccessible_False,WheelchairAccessible_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,HasTV_False,HasTV_True,RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',DogsAllowed_False,DogsAllowed_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,HappyHour_False,HappyHour_True,BikeParking_False,BikeParking_True,WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid',ByAppointmentOnly_False,ByAppointmentOnly_True,CoatCheck_False,CoatCheck_True,"BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday

In [ ]:
df_final[df_final.business_id == "yPSejq3_erxo9zdVYTBnZA"]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',Caters_False,Caters_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_True,RestaurantsTableService_False,RestaurantsTableService_True,GoodForKids_False,GoodForKids_True,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,WheelchairAccessible_False,WheelchairAccessible_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,HasTV_False,HasTV_True,RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',DogsAllowed_False,DogsAllowed_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,HappyHour_False,HappyHour_True,BikeParking_False,BikeParking_True,WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid',ByAppointmentOnly_False,ByAppointmentOnly_True,CoatCheck_False,CoatCheck_True,"BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': True, 'friday

In [ ]:
df_final.shape

(767, 177)